## Import Modules

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import itertools
import os
import zipfile
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Check for GPU

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
print(physical_devices)

tf.config.set_visible_devices(physical_devices[0], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Download cars dataset and split into 3 datasets: training, validation, testing

In [ ]:
!wget --no-check-certificate \
    https://stanfordmedicine.box.com/shared/static/1onms226m6z9oyzsgqjn93h80rqyeqhq.zip \-O /tmp/cars.zip

local_zip = '/tmp/cars.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_path = '/tmp/cars'
train_path = os.path.join(base_path, 'train')
valid_path = os.path.join(base_path, 'valid')
test_path = os.path.join(base_path, 'test')

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['BMW','Bugatti','Lamborghini','McLaren','Volkswagen'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['BMW','Bugatti','Lamborghini','McLaren','Volkswagen'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['BMW','Bugatti','Lamborghini','McLaren','Volkswagen'], batch_size=10, shuffle=False)

## Build and Train NN Model, first few layers inspired from VGG-16 fine-tuned model

In [ ]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3),activation='relu',padding='same', input_shape=(224,224,3), kernel_regularizer = 'l2'),
    Conv2D(filters=64,kernel_size=(3,3), activation='relu',padding='same', kernel_regularizer = 'l2'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=128,kernel_size=(3,3), activation='relu',padding='same', kernel_regularizer = 'l2'),
    Conv2D(filters=128,kernel_size=(3,3), activation='relu',padding='same', kernel_regularizer = 'l2'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(units=5,activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

#copy and paste this for regularization: kernel_regularizer = 'l2'

#vgg16_model = tf.keras.applications.vgg16.VGG16()
#model = Sequential()
#for layer in vgg16_model.layers[:-1]:
#    model.add(layer)
#
#for layer in model.layers:
#    layer.trainable = False

#model.add(Dense(units=5, activation='softmax'))
#model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(x=train_batches,validation_data=valid_batches,epochs=15,verbose=2)

## Conduct Testing of fitted NN parameters using the testing dataset

In [ ]:
y_test = test_batches.classes

output_layer = model.predict(x=test_batches, verbose=0)
np.round(output_layer)

y_pred = np.argmax(output_layer, axis=-1)

#Plot confusion matrix using test and prediction data
cm = confusion_matrix(y_test,y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['BMW','Bugatti','Lamborghini','McLaren','Volkswagen'])

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
#Evaluate metrics and plot validation error and training error to spot overfit possibility
loss,acc = model.evaluate(x=test_batches)

plt.style.use('ggplot')
plt.plot(hist.history['loss'],label='Train Loss')
plt.plot(hist.history['val_loss'], label='val loss')
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Losses")
#plt.xlim(0,10) #Adjust to any number suitable for plot
#plt.ylim(0,1) #Adjust to any number suitable for plot
plt.legend()
plt.show()

In [ ]:
#Plot Model accuracy vs Training accuracy as a function of epochs

plt.plot(hist.history['accuracy'],label='Train_Acc')
plt.plot(hist.history['val_accuracy'],label='Val_Acc')
plt.title('Training vs Validation Accuracies')
plt.xlabel("Epochs")
plt.ylabel("Accuracies")
plt.legend()
plt.show()